## Final-version of Accuracy Experiments

In [ ]:
#@title Setup for COLAB Environment ONLY

## mount google drive
from google.colab import drive
drive.mount('/content/drive')

## load data to local
# from google.colab import files"
# dataset_files = files.upload()

## install and upgrate necessary packages
import sys
!{sys.executable} -m pip install drive/MyDrive/pyds &> /dev/null
!{sys.executable} -m pip install --upgrade imblearn &> /dev/null
!{sys.executable} -m pip install --upgrade matplotlib &> /dev/null
!{sys.executable} -m pip install --upgrade filterpy

## add mature code to PATH
sys.path.append('/content/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     |████████████████████████████████| 177 kB 10.0 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=bdef1d32573826350d5bbc86ef3148eaa487591c3e1dcd98a1fe7d73f28f4924
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy


In [ ]:
## Import essential packages for IdAuth
%matplotlib inline

import os
import json
from collections import Counter
import numpy as np
import pandas as pd
import imblearn.over_sampling

import tensorflow as tf

from idauth.datasets.hmog import HMOGProvider
from idauth.authenticators.touch import TouchAuth
from idauth.authenticators.gait import DeepGaitExtractionModel, DeepGaitAuthenticator, DeepGaitModel
from idauth.utils.sliding_window import SlidingWindow
from idauth.utils.helpers import balance_dataset, sparse_label_converter
from idauth.utils.metrics import eer
from idauth.simulator.imposter import ImposterProvider
from idauth.simulator.timeline import Timeline
from idauth.storage import BiometricStorage, SCHEME_TIME_SEQ_10

from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, f1_score,auc,roc_curve, accuracy_score
import matplotlib.pyplot as plt

import logging



## Notebook Global Configuration

You might want to change some configurations for the notebook. Basically, you can determine the logging information to be displayed in the notebook.

In [ ]:
## GLOBAL CONFIGURATION
LOG_OUTPUT_PATH = ""
LOG_FILENAME = 'c05.log'
logging.basicConfig(format='%(asctime)-15s %(message)s',
                    filename=os.path.join(LOG_OUTPUT_PATH, LOG_FILENAME),
                    level=logging.DEBUG)
logger = logging.getLogger('simulator')

SIMULATION_RESULT_DIR = "simulation"
GAME_DETAILS_ENABELD = False # to show training and testing sessions
VISUABLIZATION_ENABLED = False # show a timeline figure to visualize decisions

## Helper Functions (For HMOG)

```simple_merge_acc_gyro```: merge accelerometer and gyroscope data in a non-temporal manner. ```step``` is the downsamplingr rate. E.g., step = 2, original frequency = 100Hz, then new frequency = 50Hz.

In [ ]:
def simple_merge_acc_gyro(acc_df, gyro_df, step=2):
    size = min(acc_df.shape[0], gyro_df.shape[0])
    acc = acc_df.iloc[:size][['x', 'y', 'z']].to_numpy()
    gyro = gyro_df.iloc[:size][['x', 'y', 'z']].to_numpy()
    merged = np.concatenate((acc, gyro), axis=1)
    return merged[::step]

def touch_df_to_features(df):
    raw = df[['swipe_id', 'time', 'x', 'y', 'pressure', 'area', 'orientation',
              'activity']].values
    features = TouchAuth().extract_features(raw) # to be staticmethod
    return features

def get_touch_features_by_user_session(provider: HMOGProvider,
                                       user_session_settings):
    """
    Obtain a dictionary like:"
      result['<user_name>'][<session_id>]: feature list
    """
    result = {}
    for user, sessions in user_session_settings.items():
        result[user] = {}
        for sess in sessions:
            features = touch_df_to_features(provider.data[user][sess]['touch'])
            result[user][sess] = features
    return result

def get_gait_features_by_user_session(provider: HMOGProvider,
                                      user_session_settings):
    result = {}
    for user, sessions in user_session_settings.items():
        result[user] = {}
        for sess in sessions:
            tmp_data = simple_merge_acc_gyro(hmog.data[user][sess]['acc'],
                                         hmog.data[user][sess]['gyro'])
            features = DeepGaitAuthenticator().extract_features(tmp_data)
            result[user][sess] = features
    return result

def get_gait_features(provider, user, session):
    tmp_data = simple_merge_acc_gyro(provider.data[user][session]['acc'],
                                     provider.data[user][session]['gyro'])
    features = DeepGaitAuthenticator().extract_features(tmp_data)
    return features

def get_touch_features(provider, user, session):
    features = touch_df_to_features(provider.data[user][session]['touch'])
    return features

def get_features(provider, auth_type, user, session):
    if auth_type == 'gait':
        return get_gait_features(provider, user, session)
    elif auth_type == 'touch':
        return get_touch_features(provider, user, session)

###
## Timeline operations
###

def get_events_for_user_session(user, session, new_basetime):
    logger.info("generate events for: " + str(user) + ", " + str(session))
    base_time = int(min(hmog.data[user][session]['acc'].iloc[0]['time'],
                         hmog.data[user][session]['gyro'].iloc[0]['time']))
    # e.g. new_basetime starts from 0, base_time = 1000, t0 = 1003
    # offset = - base_time + new_basetime
    # offset = - 1000 + 0
    # new_timestamp = t0 + offset = 1003 - 1000 = 3
    offset = new_basetime - base_time
    acc_events = get_acc_events_for_user_session(user, session, offset)
    gyro_events = get_gyro_events_for_user_session(user, session, offset)
    touch_events = get_touch_events_for_user_session(user, session, offset)
    merged_events = sorted(acc_events + gyro_events + touch_events)
    return merged_events

def get_events_for_user_session_with_limit(user, session, new_basetime, limit):
    logger.info("generate events for: " + str(user) + ", " + str(session))
    base_time = int(min(hmog.data[user][session]['acc'].iloc[0]['time'],
                        hmog.data[user][session]['gyro'].iloc[0]['time']))
    end_time = base_time + limit
    # e.g. new_basetime starts from 0, base_time = 1000, t0 = 1003
    # offset = - base_time + new_basetime
    # offset = - 1000 + 0
    # new_timestamp = t0 + offset = 1003 - 1000 = 3
    offset = new_basetime - base_time
    acc_events = get_acc_events_for_user_session(user, session, offset, end_time)
    gyro_events = get_gyro_events_for_user_session(user, session, offset, end_time)
    touch_events = get_touch_events_for_user_session(user, session, offset, end_time)
    merged_events = sorted(acc_events + gyro_events + touch_events)
    return merged_events


def get_acc_events_for_user_session(user, session, offset, end_time=None, start_time=None):
    df = hmog.data[user][session]['acc'].iloc[::2]
    result = []
    for index, row in df.iterrows():
        if start_time is not None and int(row['time']) < start_time:
            continue
        if end_time is not None and int(row['time']) > end_time:
            break
        result.append((int(row['time']) + offset, 'acc',
                       (row['x'], row['y'], row['z'])))
    return result

def get_gyro_events_for_user_session(user, session, offset, end_time=None, start_time=None):
    df = hmog.data[user][session]['gyro'].iloc[::2]
    result = []
    for index, row in df.iterrows():
        if start_time is not None and int(row['time']) < start_time:
            continue
        if end_time is not None and int(row['time']) > end_time:
            break
        result.append((int(row['time']) + offset, 'gyro',
                       (row['x'], row['y'], row['z'])))
    return result

def get_touch_events_for_user_session(user, session, offset, end_time=None, start_time=None):
    df = hmog.data[user][session]['touch']
    result = []
    for index, row in df.iterrows():
        if start_time is not None and int(row['time']) < start_time:
            continue
        if end_time is not None and int(row['time']) > end_time:
            break
        result.append((int(row['time']) + offset, 'touch',
                       (row['swipe_id'], row['time'] + offset, row['x'], row['y'],
                        row['pressure'], row['area'], row['orientation'], row['activity'])))
    return result



## Scoring functions

In [ ]:
## Kalman Filter based fusion

from filterpy.kalman import KalmanFilter


class KFScorer:

    def add(self, z):
        self.kf.predict()
        # print(z)
        if len(z) == 0:
            self.kf.update(self.last_z)
        else:
            self.kf.update(z)

        self.last_z = z
        return self.kf.x

    def init(self, x0):
        self.kf.x = x0

    def setR(self, R):
        self.kf.R = R

    def __init__(self, n_classes, n_auth, discount, Q=0.25):
        self.kf = KalmanFilter(dim_x=n_classes, dim_z=n_auth)
        self.kf.Q = np.eye(n_classes) * Q
        self.kf.H = np.ones([n_auth, n_classes])
        self.kf.F = np.eye(n_classes) * discount
        self.last_z = np.zeros(n_auth)

In [ ]:
## DS FUNC
import pyds as ds
import functools

TIME_INTERVAL = 100000

def ds_fusion_2(scores, uncertainties):
    mass_groups = [ds.MassFunction({'a': s * (1 - u), 'b': (1 - s) * (1 - u),
                                    'ab': u})
                   for s, u in zip(scores, uncertainties)]
    combined_mass = functools.reduce(lambda a, b: a & b, mass_groups)
    return combined_mass['a']


# =======================================
#  Scoring Strategy
# =======================================

def gait_scoring(buffer, t0):
    raw = [data for auth_type, data, t in buffer
           if t == t0 and auth_type == 'gait']
    return raw

def touch_scoring(buffer, t0):
    raw = [data for auth_type, data, t in buffer
           if t == t0 and auth_type == 'touch']
    return raw

def plain_average_scoring(buffer, t0, time_interval=TIME_INTERVAL):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    # print(sel_buffer)
    auth_types, results, times = zip(*sel_buffer)
    return [np.array(np.average(results, axis=0))]


def weighted_average_eer_scoring(buffer, t0, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    # print(sel_buffer)

    weighted_scores = [[r * max(0, 1 - 2*strategy[auth_type]['eer'][i])
                        for i, r in enumerate(result)]
                        for auth_type, result, time in sel_buffer]

    return [np.array(np.average(weighted_scores, axis=0))]


def weighted_average_auc_scoring(buffer, t0, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    weighted_scores = [[r * min(1,  strategy[auth_type]['auc'][i])
                        for i, r in enumerate(result)]
                        for auth_type, result, time in sel_buffer]

    return [np.array(np.average(weighted_scores, axis=0))]

def deer_ds_scoring(buffer, t0, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
        uncertainty = [min(1, 2*strategy['gait']['eer'][i]),
                       min(1, 2*strategy['touch']['eer'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    return [np.array(final_result)]


def redeer_ds_scoring(buffer, t0, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
        uncertainty = [max(0, 1 - 2 * strategy['gait']['eer'][i]),
                       max(0, 1 - 2 * strategy['touch']['eer'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    return [np.array(final_result)]

def auc_ds_scoring(buffer, t0, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
        uncertainty = [max(0, 1 - strategy['gait']['auc'][i]),
                       max(0, 1 - strategy['touch']['auc'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    return [np.array(final_result)]


def kf_pu_scoring(buffer, t0, kmfs, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    result = []
    for u, kmf in enumerate(kmfs):
        # per user kalman filter
        raw_r = np.eye(2)
        obs = np.copy(kmf.last_z)
        for i, auth in enumerate(['touch', 'gait']):
            raw_r[i][i] = strategy[auth]['eer'][u]
            if auth in modalities:
                # print(obs[i], modalities[auth])
                obs[i] = np.average(modalities[auth], axis=0)[u]
        kmf.setR(raw_r)
        result.append(kmf.add(obs))

    return [np.array(result)]

def kalman_filter_scoring(buffer, t0, kmf, time_interval=TIME_INTERVAL, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    # update R function
    raw_r = np.eye(len(modalities.keys()))
    for i, auth in enumerate(['touch', 'gait']):
        raw_r[i][i] = strategy[auth]['accuracy']

    kmf.setR(raw_r)

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    result = kmf.add(np.array([result_touch, result_gait]))
    return [result]





SUPPORTED_SCORER = {
    'simple_average': plain_average_scoring,
    'weighted_average_eer': weighted_average_eer_scoring,
    'weighted_average_auc': weighted_average_auc_scoring,
    # 'dempster_shafer': dempster_shafer_score,
    # 'redeer_dempster_shafer': redeer_ds_scoring,
    'deer_dempster_shafer': deer_ds_scoring,
    'auc_dempster_shafer': auc_ds_scoring,
    'kalman_filter': kf_pu_scoring,
    # 'kalman_filter_0.1_0.9': kf_pu_scoring,
    # 'kalman_filter_0.01_0.9': kf_pu_scoring,
    # 'kalman_filter_0.001_0.9': kf_pu_scoring,
    # 'kalman_filter_0.25_1.0': kf_pu_scoring,
    # 'kalman_filter_0.25_0.8': kf_pu_scoring,
    # 'kalman_filter_0.25_0.7': kf_pu_scoring,
    # 'fixed_dempster_shafer': fixed_dempster_shafer_score
}


# =======================================
#  Different decision making strategies
# =======================================


def gait_only(buffer, t0):
    raw = [data for auth_type, data, t in buffer
           if t == t0 and auth_type == 'gait']

    if len(raw) < 3:
        return -1

    result = np.argmax(raw, axis=1)
    ctr = Counter(result)
    # print(ctr.most_common()[0])
    return ctr.most_common()[0][0]


def touch_only(buffer, t0):
    result = np.argmax([data for auth_type, data, t in buffer
                        if auth_type == 'touch'],
                        axis=1)
    to_check = result[-7:]
    final_result = -1
    sw = SlidingWindow(7, 4)
    for i in to_check:
        final_result = sw.add(i)
    return final_result


def direct_score(buffer, t0, time_interval=30000):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    result = np.argmax(np.average(results, axis=0))

    return result

def direct_decision(buffer, t0, time_interval=30000):
    result = [data for auth_type, data, t in buffer
              if np.abs(t - t0) < time_interval]
    ctr = Counter(result)
    item = ctr.most_common()
    if len(item) <= 0:
        return -1
    else:
        return ctr.most_common()[0][0]



def deer_dempster_shafer_score(buffer, t0, time_interval=30000, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return np.argmax(np.average(modalities[list(modalities.keys())[0]], axis=0))

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
#         uncertainty = [uncertainty_eer(strategy['gait']['eer'][i]),
#                        uncertainty_eer(strategy['touch']['eer'][i])]
        uncertainty = [min(1, 2*strategy['gait']['eer'][i]),
                       min(1, 2*strategy['touch']['eer'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    # result = np.argmax(np.average(results, axis=0))
    return np.argmax(final_result)



SUPPORTED_DECISION_MAKER = {
    'simple_average': direct_score,
    'simple_voting': direct_decision,
    # 'weight_average': weighted_score,
    # 'dempster_shafer': dempster_shafer_score,
    # 'wrong_dempster_shafer': wrong_dempster_shafer_score,
    'deer_dempster_shafer': deer_dempster_shafer_score,
    # 'fixed_dempster_shafer': fixed_dempster_shafer_score
}


## Simulator Code


In [ ]:
GAIT_NO_GAITS_DETECTED = -101
GAIT_GAITS_DETECTED = 101
GAIT_INSUFFICIENT_STEPS = -100

TOUCH_NO_SWIPE_DETECTED = -201
TOUCH_SWIPE_DETECTED = 201

DECISION_INSUFFICIENT_DATA = -1

EVENT_DEVICE_SHARING = "device sharing"

### comment types
COMMENT_FALSE_REJECTION = "false_rejection"
COMMENT_FALSE_ACCEPTANCE = "false_acceptance"
COMMENT_FALSE_MISCLASSIFICATION = "false_classification"
COMMENT_CONFIRM_USER_CHANGE = "confirm_user_change"

### special events
EVENT_SESSION_START = "session_start"
EVENT_SESSION_END = "session_end"
EVENT_ACCESS_GRANT = "access_grant"
EVENT_TRAIN_MODELS = "train_models"
EVENT_USER_CHANGE = "user_change" # user change signal is only for ground truth
EVENT_ADD_USER = "add_user"

SPECIAL_EVENTS = [
    EVENT_SESSION_START,
    EVENT_SESSION_END,
    EVENT_ACCESS_GRANT,
    EVENT_TRAIN_MODELS
]

### system states
STATE_ENROLLMENT = "state_enrollment"
STATE_NORMAL = "state_normal"
STATE_SHARING = "state_sharing"

SYSTEM_STATES = [
    STATE_ENROLLMENT,
    STATE_NORMAL,
    STATE_SHARING
]

### user states
USER_UNKNOWN = "Unknown"
USER_STRANGER = "Stranger"

### reaction types
REACTION_NONE = 0
REACTION_REJECT = -1
REACTION_USER_CHANGE = 9
REACTION_ACCEPT = 1



class TestPlayer:

    def train_auth(self, auth_type):
        auth = self.auth[auth_type]
        settings = self.gen_train_test_set(auth_type)
        # print('train', auth_type, settings['train_data'].shape, np.max(settings['train_label']))
        # print('test', auth_type, settings['test_data'].shape, np.max(settings['test_label'])
        reports = auth.train_model_with_eval((settings['train_label'],
                                              settings['train_data']),
                                             (settings['test_label'],
                                              settings['test_data']))

        # print(reports)
        return reports

    def train_all(self):
        for name, auth in self.auth.items():
            self.auth_report[name] = self.train_auth(name)
            print(name)
            print(self.auth_report[name]['report'])

        self.kalman = []
        for i in range(len(self.user_profile) + 1):
            kf = KFScorer(1,
                          len(self.auth), 0.9)
            kf.init(0.5)
            self.kalman.append(kf)

    def gen_train_test_set(self, auth_type):
        train_data_set = []
        train_label_set = []
        test_data_set = []
        test_label_set = []

        train_test_dict = self.storage.get_train_test_sets(auth_type, SCHEME_TIME_SEQ_10)
        for key, tt in train_test_dict['train'].items():
            label = self.get_user_label(key)
            train_data_set.append(tt)
            train_label_set.append(np.ones(tt.shape[0], dtype=int) * label)
        for key, tt in train_test_dict['test'].items():
            label = self.get_user_label(key)
            test_data_set.append(tt)
            test_label_set.append(np.ones(tt.shape[0], dtype=int) * label)

        # sample test dataset
        train_label, train_data = balance_dataset(np.concatenate(train_label_set),
                                                  np.concatenate(train_data_set),
                                                  1,
                                                  len(self.user_profile),
                                                  0)
        test_label, test_data = balance_dataset(np.concatenate(test_label_set),
                                                np.concatenate(test_data_set),
                                                1,
                                                len(self.user_profile),
                                                0)
        # print(test_label.shape, test_data.shape, train_data_set[0].shape,
        #       train_label_set[0].shape)
        # print(test_label)
        return {
            'train_data': train_data, # np.concatenate(train_data_set),
            'train_label': train_label, # np.concatenate(train_label_set),
            'test_data': test_data,
            'test_label': test_label
        }

    def feedback(self, comment):
        """
        feedback function receives user's feedback or other signal
        one representative example is: user correct misclassification
        comment format:
        (COMMENT_TYPE, DETAILS)
        """
        com_type, com = comment
        if com_type == COMMENT_FALSE_REJECTION:
            self.current_user = com
        elif com_type == COMMENT_FALSE_MISCLASSIFICATION:
            self.current_user = com
        elif com_type == COMMENT_CONFIRM_USER_CHANGE:
            # only after getting confirmation then update cached data to storage
            self.update_cache_to_storage(self.current_user)



    def __handle_special_events(self, event):
        if event == EVENT_SESSION_START:
            # handle a session start
            ## TODO: tasks that need to be done at session start
            pass

        elif event == EVENT_SESSION_END:
            # handle a session end
            if self.status == STATE_SHARING:
                self.status = STATE_NORMAL

            # assign allocation retrain
            # print("session end, train model")
            if self.flag_store_history:
                self.update_cache_to_storage(self.current_user)
            # self.train_all()

            # reset current user to unknown
            self.current_user = USER_UNKNOWN
            self.previous_user = USER_UNKNOWN

            # reset buffer and decisions
            self.buffer = {
                'acc': [],
                'gyro': [],
                'touch': []
            }
            self.decision_sw.reset()
            self.last_update = {
                'acc': -1,
                'gyro': -1,
                'touch': -1
            }

            self.detect_ctr = {
                'gait': 0
            }

            self.cur_swipe = -1
            self.decision_buffer = []
            self.another_buffer = []


        elif event == EVENT_ACCESS_GRANT:
            # handle a device sharing
            self.set_status(STATE_SHARING)

        elif event == EVENT_TRAIN_MODELS:
            self.train_all()


    def dispatch(self, event):
        time, sensor, data = event

        if sensor in SPECIAL_EVENTS:
            self.__handle_special_events(sensor)
            return REACTION_NONE

        self.buffer[sensor].append(data)

        reaction = REACTION_NONE # default reaction is nothing

        ## gait checker
        if sensor == 'acc':
            if len(self.buffer['acc']) >= 512 \
              and len(self.buffer['gyro']) >= 512 \
              and self.detect_ctr['gait'] >= 256:
                result = self.gait_detection()
                if result['code'] == GAIT_GAITS_DETECTED:
                    decision = self.aggregate("gait", result['result'], time)
                    if decision >= 0:
                        reaction = self.react_to_decision(decision)
                self.detect_ctr['gait'] = 0
            else:
                self.detect_ctr['gait'] += 1


        if sensor == 'touch':
            if self.cur_swipe == -1:
                self.cur_swipe = data[0]
            elif data[0] != self.cur_swipe:
                result = self.touch_detection()
                self.cur_swipe = data[0]
                if result['code'] == TOUCH_SWIPE_DETECTED:
                    decision = self.aggregate("touch", result['result'], time)
                    if decision >= 0:
                        reaction = self.react_to_decision(decision)
        return reaction


    def react_to_decision(self, decision):
        if decision < 0:
            # invalid decision
            return REACTION_NONE

        if decision == 0:
            self.current_user = USER_STRANGER
            if self.status == STATE_SHARING:
                return REACTION_NONE

            return REACTION_REJECT

        if self.current_user == USER_UNKNOWN:
            self.current_user = self.user_profile[decision - 1]
            return REACTION_ACCEPT
        else:
            if self.current_user != self.user_profile[decision - 1]:
                # TODO: react to user change

                self.previous_user = self.current_user
                self.current_user = self.user_profile[decision - 1]
                return REACTION_USER_CHANGE
            else:
                return REACTION_ACCEPT

    def cache_auth_data(self, auth_type, prediction, features):
        self.biometric_cache[auth_type].append((prediction, features))

    def update_cache_to_storage(self, true_user, crop=None):
        if true_user not in self.user_profile:
            print("current session does not belong to a valid user, discarded")
            self.clear_cache()
            return

        for auth_type, cached_data in self.biometric_cache.items():
            if len(cached_data) == 0:
                continue
            _, features = zip(*cached_data)
            self.storage.add_session_data(true_user,
                                          auth_type,
                                          np.array(features))
        self.clear_cache()

    def clear_cache(self):
        self.biometric_cache = {
            'gait': [],
            'touch': []
        }

    def gait_detection(self):
        dga = self.auth['gait']
        acc = np.array(self.buffer['acc'][-512:])
        gyro = np.array(self.buffer['gyro'][-512:])
        merged = np.concatenate([acc, gyro], axis=1)
        # processed = np.array([merged]).reshape(1, 1024, 6, 1)
        auth_result = None
        features = dga.extract_features(merged)
        if features is None:
            return {
                'code': GAIT_NO_GAITS_DETECTED,
                'result': None
            }
        else:
            r = dga.authenticate(features)
            for feature, result in zip(features, r):
                self.cache_auth_data('gait', result, feature)

            auth_result = r
        if auth_result is None or len(auth_result) == 0:
            return {
                'code': GAIT_NO_GAITS_DETECTED,
                'result': None
            }
        else:

            return {
                'code': GAIT_GAITS_DETECTED,
                'result': auth_result.tolist()
            }

    def gait_detection_full(self):
        dga = self.auth['gait']
        acc = np.array(self.buffer['acc'][-1024:])
        gyro = np.array(self.buffer['gyro'][-1024:])
        merged = np.concatenate([acc, gyro], axis=1)
        processed = np.array([merged]).reshape(1, 1024, 6, 1)
        result = dge.predict(processed.transpose(0, 2, 1, 3))[0]
        gaits = []
        st = 0
        flag = False
        tolerant_ctr = 0
        max_tolerance = 5
        for i in range(result.shape[0]):
            if result[i] > 0.5:
                if not flag:
                    st = i
                    flag = True
                tolerant_ctr = 0
            elif flag:
                if tolerant_ctr >= max_tolerance:
                    if i - st > 128:
                        gaits.append((st, i))
                        flag = False
                    else:
                        tolerant_ctr = 0
                        flag = False
                else:
                    tolerant_ctr += 1
        if flag and result.shape[0] - 1 - st > 128:
            gaits.append((st, result.shape[0] - 1))

        if len(gaits) == 0:
            return {
                'code': GAIT_NO_GAITS_DETECTED,
                'result': None
            }

        auth_result = None
        for gait in gaits:
            features = dga.extract_features(merged[gait[0]: gait[1] + 1])
            if features is None:
                continue
            else:
                r = dga.authenticate(features)
                for feature, result in zip(features, r):
                    self.cache_auth_data('gait', result, feature)

                auth_result = r if auth_result is None else np.concatenate((auth_result, r), axis=0)
        if auth_result is None or len(auth_result) == 0:
            return {
                'code': GAIT_NO_GAITS_DETECTED,
                'result': None
            }
        else:

            return {
                'code': GAIT_GAITS_DETECTED,
                'result': auth_result.tolist()
            }


    def touch_detection(self):
        toa = self.auth['touch']
        features = toa.extract_features(self.buffer['touch'])
        if len(features) == 0:
            return {
                'code': TOUCH_NO_SWIPE_DETECTED,
                'result': None
            }
        else:
            result = toa.authenticate([features[-1]])
            self.cache_auth_data('touch', result, features[-1])
            return {
                'code': TOUCH_SWIPE_DETECTED,
                'result': result
            }

    def aggregate(self, auth_type, raw_result, time):
        self.decision_buffer += [(auth_type, i, time) for i in raw_result]


        ### record all decision scores
        if auth_type == 'gait':
            result = gait_scoring(self.decision_buffer, time)
            # print(auth_type, result)
            # self.decision_scores[auth_type] += [(time, r) for r in result]
            self.decision_scores[auth_type].append((time, np.average(result, axis=0)))

        if auth_type == 'touch':
            result = touch_scoring(self.decision_buffer, time)
            self.decision_scores[auth_type] += [(time, r) for r in result]

        for dem_name, dem_maker in SUPPORTED_SCORER.items():
            if dem_name == 'simple_voting':
                result = dem_maker(self.another_buffer, time)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            elif 'weighted_average' in dem_name:
                result = dem_maker(self.decision_buffer, time,
                                   strategy=self.auth_report)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            elif 'dempster_shafer' in dem_name:
                result = dem_maker(self.decision_buffer, time,
                                   strategy=self.auth_report)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            elif 'kalman_filter' in dem_name:
                result = dem_maker(self.decision_buffer, time,
                                   self.kalman, strategy=self.auth_report)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            else:
                result = dem_maker(self.decision_buffer, time)
                self.decision_scores[dem_name] += [(time, r) for r in result]


        if auth_type == 'gait':
            result = gait_only(self.decision_buffer, time)
            if result >= 0:
                self.detection_events['gait'].append((time, result))
                self.another_buffer.append((auth_type, result, time))

        if auth_type == 'touch':
            result = touch_only(self.decision_buffer, time)
            if result >= 0:
                self.detection_events['touch'].append((time, result))
                self.another_buffer.append((auth_type, result, time))


        # GO THROUGH EACH DECISION MAKING FUNCTIONS
        # TODO: THE FOLLOWING CODE ONLY KEEPS FINAL RESULTS

        result = -1
        for dem_name, dem_maker in SUPPORTED_DECISION_MAKER.items():
            if dem_name == 'simple_voting':
                result = dem_maker(self.another_buffer, time)
            elif 'dempster_shafer' in dem_name:
                result = dem_maker(self.decision_buffer, time,
                                   strategy=self.auth_report)
            elif dem_name == 'weight_average':
                result = dem_maker(self.decision_buffer, time,
                                   weight_scheme = {
                                       'gait': self.auth_report['gait']['accuracy'],
                                       'touch': self.auth_report['touch']['accuracy'],
                                   })
            else:
                result = dem_maker(self.decision_buffer, time)
            if result >= 0:
                self.detection_events[dem_name].append((time, result))

        return self.decision_sw.add(self.detection_events['deer_dempster_shafer'][-1])

    # =====================================
    # System status management functions
    # =====================================
    def set_status(self, state):
        assert state in SYSTEM_STATES
        self.status = state

    def reset_authenticators(self):
        self.auth = {
            'touch': TouchAuth(),
            'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
        }


    # =====================================
    # User management functions
    # =====================================
    def add_user(self, user_id):
        if user_id in self.user_profile:
            print("User already registered.")
            return
        self.user_profile.append(user_id)
        self.storage.add_user(user_id)

    def get_user_label(self, user_id):
        # generate user label dict
        user_label_dict = {}
        for i, user in enumerate(self.user_profile):
            user_label_dict[user] = i + 1
        if user_id not in user_label_dict:
            return 0
        else:
            return user_label_dict[user_id]

    def enroll(self, user_id, details):
        self.add_user(user_id)
        for item in details:
            # print(item['user'], user_id)
            assert item['user'] == user_id
            self.storage.add_session_data(user_id, item['type'], item['feature'])
        self.reset_authenticators()
        self.train_all()


    # =====================================
    # Utility functions for experiments
    # =====================================

    def save_decision(self, dem_name, out_path):
        with open(os.path.join(out_path, dem_name + ".txt"), 'w') as fp:
            for time, result in self.detection_events[dem_name]:
                fp.write(str(time) + "," + str(result) + "\n")

    def save_all_decisions(self, out_path):
        for dem_name in SUPPORTED_DECISION_MAKER.keys():
            self.save_decision(dem_name, out_path)

    def set_flag_store_history(self, b):
        self.flag_store_history = b

    def __init__(self, authenticators, auth_report=None, imposter_set=None):
        self.auth = authenticators
        self.auth_report = {} if auth_report is None else auth_report
        self.status = STATE_NORMAL # default normal state
        self.flag_store_history = True
        self.current_user = USER_UNKNOWN
        self.previous_user = USER_UNKNOWN
        self.buffer = {
            'acc': [],
            'gyro': [],
            'touch': []
        }

        self.last_update = {
            'acc': -1,
            'gyro': -1,
            'touch': -1
        }

        self.detect_ctr = {
            'gait': 0
        }

        self.cur_swipe = -1

        self.decision_buffer = []
        self.another_buffer = []

        self.detection_events = {
            'touch': [],
            'gait': [],
        }

        for dem in SUPPORTED_DECISION_MAKER.keys():
            self.detection_events[dem] = []

        self.decision_scores = {
            'touch': [],
            'gait': [],
        }

        for dem in SUPPORTED_SCORER.keys():
            self.decision_scores[dem] = []

        ## dynamic system related variables
        self.biometric_cache = {
            'touch': [],
            'gait': []
        }

        self.storage = BiometricStorage(imposter_set=imposter_set)

        self.user_profile = []
        self.decision_sw = SlidingWindow(7, 4)

In [ ]:

# scripter runner
# to handle exceptions during a simulation
# keep record of all progress
# calculate metrics of a simulation

## basic script workflow
## 1. obtain next event from timeline
## 2. feed it into the player
## 3. obtain detection result
## 4. update simulation status
## 5. capture exceptions
## 6. feedback

## whenever gets a valid result from player, first chech if it matches groundtruth

## some important definitions
## 1. false acceptance: false acceptance should be counted at session-level,

## when the user has changed (groundtruth) but the system
## is late to reject a stranger or identify the true user


## about metrics

EXCEPTION_FALSE_ACCEPTANCE = "FA"
EXCEPTION_FALSE_REJECTION = "FN"
EXCEPTION_MISCLASSIFICATION = "MC"


def generate_enrollment_data(provider, session_info):
    user = session_info[0]
    print("gen", user)
    d_sessions = session_info[1:]
    gait_sessions = [d_sessions[i + 1] for i in range(0, len(d_sessions), 2)
                     if d_sessions[i] == 'gait']
    touch_sessions = [d_sessions[i + 1] for i in range(0, len(d_sessions), 2)
                     if d_sessions[i] == 'touch']

    combined = []
    for sess in gait_sessions:
        feature = get_features(provider, 'gait', user, sess)
        combined.append({
            'user': user,
            'type': 'gait',
            'feature': feature
        })
    for sess in touch_sessions:
        feature = get_features(provider, 'touch', user, sess)
        combined.append({
            'user': user,
            'type': 'touch',
            'feature': feature
        })
    return combined

def script_runner(timeline: Timeline, player: TestPlayer):

    ## initialize the runner status
    cur_user = USER_UNKNOWN

    session_results = []
    tmp = None
    gt_start_time = 0

    for event in timeline:
        result = REACTION_NONE
        if event[1] not in [EVENT_USER_CHANGE, EVENT_ADD_USER]:
            # EVENT_USER_CHANGE IS NOT FOR PLAYER
            result = player.dispatch(event)

        true_user = timeline.get_groundtruth_result(event[0])


        if event[1] in [EVENT_SESSION_END, EVENT_USER_CHANGE]:
            logger.info("[groundtruth] detect session end")
            # print("tmp", tmp)
            processed = process_session_result(tmp)
            if processed is not None:
                session_results.append(processed)
            tmp = None
        if event[1] in [EVENT_SESSION_START, EVENT_USER_CHANGE]:
            logger.info("[groundtruth] detect session start")
            tmp = {
                'num_detection': 0,
                'num_accept': 0,
                'num_reject': 0,
                'num_fp': 0,
                'num_fn': 0,
                'num_mc': 0, # misclassification
                'delay': -1,
                'exceptions': []
            }
            gt_start_time = event[0]
        if event[1] == EVENT_ADD_USER:
            ## explicit enrollment format
            ## name, session
            logger.info("[new user enrolment] user: " + str(event[2][0]))
            enroll_data = generate_enrollment_data(hmog, event[2])
            player.enroll(event[2][0], enroll_data)

        ## possible results:
        ## reaction types
        ## REACTION_NONE = 0
        ## REACTION_REJECT = -1
        ## REACTION_USER_CHANGE = 9
        ## REACTION_ACCEPT = 1

        if result == REACTION_ACCEPT:

            ## compare to ground truth
            tmp['num_detection'] += 1
            tmp['num_accept'] += 1
            if player.current_user != true_user:
                # current user
                if true_user not in player.user_profile:
                    # false acceptance
                    tmp['num_fp'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_FALSE_ACCEPTANCE))
                    player.feedback((COMMENT_FALSE_ACCEPTANCE, USER_STRANGER))
                else:
                    # misclassification
                    tmp['num_mc'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_MISCLASSIFICATION))
                    player.feedback((COMMENT_FALSE_MISCLASSIFICATION, true_user))
            else:
                # true positive
                # check if it is the first detection
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time

        elif result == REACTION_REJECT:
            tmp['num_detection'] += 1
            tmp['num_reject'] += 1
            if true_user in player.user_profile:
                # false rejection
                tmp['num_fn'] += 1
                tmp['exceptions'].append((event[0], EXCEPTION_FALSE_REJECTION))
                player.feedback((COMMENT_FALSE_REJECTION, true_user))
            else:
                # true rejection (check first detection)
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time

        elif result == REACTION_USER_CHANGE:
            tmp['num_detection'] += 1
            # check if it is really a user change
            tmp['num_accept'] += 1
            if true_user == player.current_user:
                # yes!
                player.feedback((COMMENT_CONFIRM_USER_CHANGE, None))
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time
                else:
                    print("there must be something going wrong")
            else:
                if true_user not in player.user_profile:
                    # false acceptance
                    tmp['num_fp'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_FALSE_ACCEPTANCE))
                    player.feedback((COMMENT_FALSE_ACCEPTANCE, USER_STRANGER))
                else:
                    tmp['num_mc'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_MISCLASSIFICATION))
                    player.feedback((COMMENT_FALSE_MISCLASSIFICATION, true_user))

    return session_results


def process_session_result(tmp):
    if tmp is None:
        return None
    return tmp

# Evaluation Code Snippets

In [ ]:
def gen_groundtruth_for_decision_scores(tml: Timeline, player: TestPlayer):
    # generate groundtruth table for decision scores
    # return {<decison_maker>: {'pred': <scores>, 'true': <groundtruth labels>}}
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        # process for a certain dec
        grt_list = []
        pred_list = []
        for time, result in dec_list:
            user = tml.get_groundtruth_result(time)
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            grt_list.append(grt)
            pred_list.append(result)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list
        }
    return result_dict


def one_vs_rest_binarize(decision_pair, pos_loc):
    score = np.array(decision_pair['pred'])[:, pos_loc]
    grt = (np.array(decision_pair['true']) == pos_loc).astype(int)
    full_score = np.array([[1 - s, s] for s in score])
    return {
        'score': score,
        'true': grt,
        'pred': full_score
    }

def one_vs_one_binarize(decision_pair, pos_loc, neg_loc):
    score = np.array(decision_pair['pred'])
    truth = np.array(decision_pair['true'])
    sel_score = score[(truth == pos_loc) | (truth == neg_loc)]
    sel_truth = truth[(truth == pos_loc) | (truth == neg_loc)]
    grt = (sel_truth == pos_loc).astype(int)
    single_score = np.array([s[pos_loc] for s in sel_score])
    full_score = np.array([[s[neg_loc], s[pos_loc]] for s in sel_score])
    return {
        'score': single_score,
        'true': grt,
        'pred': full_score
    }


def gen_per_segment_decision(tml: Timeline, player: TestPlayer):
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        grt_list = []
        pred_list = []
        time_list = []
        segment_no = 0
        st, ed, user, _ = tml.user_groundtruth[segment_no]
        seg_grt = []
        seg_pred = []
        seg_time = []
        for time, result in dec_list:
            # user = tml.get_groundtruth_result(time)
            assert time >= st
            if time > ed:
                # move to the next segment
                grt_list.append(seg_grt)
                pred_list.append(seg_pred)
                time_list.append(seg_time)
                seg_grt = []
                seg_pred = []
                seg_time = []
                segment_no += 1
                assert segment_no <= len(tml.user_groundtruth)
                st, ed, user, _ = tml.user_groundtruth[segment_no]
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            seg_grt.append(grt)
            seg_pred.append(result)
            seg_time.append(time - st)
        grt_list.append(seg_grt)
        pred_list.append(seg_pred)
        time_list.append(seg_time)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list,
            'time': time_list
        }
    return result_dict



def get_metrics_per_segment(decision_pair, sliding_window=None):
    positives = 0
    negatives = 0
    false_rejections = 0
    true_rejections = 0
    false_identifications = 0
    latency_rejection = []
    result_dict = {
        'pos': 0,
        'neg': 0,
        'fr': 0,
        'tr': 0,
        'fi': 0,
        'latency': []
    }

    for i, j, t in zip(decision_pair['pred'], decision_pair['true'],
                      decision_pair['time']):
        # print(i)
        # print(j)
        score = np.array(i)
        truth = np.array(j)
        result = np.argmax(score, axis=1)
        if sum(truth) > 0:
            result_dict['pos'] += 1
            # positive case, focus on false rejection and false identification
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res in result:
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif decision != truth[0] and decision > 0:
                        print(decision, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
            else:
                for res in result:
                    if res == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif res != truth[0]:
                        print(res, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
        else:
            result_dict['neg'] += 1
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res, time in zip(result, t):
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
            else:
                for res, time in zip(result, t):
                    if res == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
    return result_dict


def gen_all_metrics_for_scheme(dcp, num_valid_users=2):
    final_result = dict()
    for user in range(num_valid_users):
        per_user_result = dict()
        result = one_vs_rest_binarize(dcp, user + 1)

        per_user_result['true'] = result['true']
        per_user_result['score'] = result['score']
        per_user_result['pred'] = result['pred']

        # obtain roc curve
        fpr, tpr, thres = roc_curve(result['true'], result['score'])

        per_user_result['fpr'] = fpr
        per_user_result['tpr'] = tpr
        per_user_result['thres'] = thres

        eer_score, _ = eer(result['true'], result['score'])
        per_user_result['eer'] = eer_score
        per_user_result['auc'] = roc_auc_score(result['true'], result['score'])
        final_result[user + 1] = per_user_result
    return final_result

def gen_all_metrics(score_dict, num_valid_users=2):
    return {
        name: gen_all_metrics_for_scheme(dcp, num_valid_users)
        for name, dcp in score_dict.items()
    }


# metrics_all = gen_all_metrics(score_dict, 2)


# fcs = FCS('simple_average')
# fcs.get_metric(result['true'], result['pred'],
#                (result['score'] > 0.3).astype(int))



import seaborn as sns

class FCS():

    def __init__(self, classifier_name):
        self.classifier_name = classifier_name
        self.true_labels = None
        self.pred_probs = None
        self.pred_labels = None
        self.plot = None
        self.axes = None

        return

    def get_metric(self, true_labels, predicted_probs, pred_labels, bins="auto",
                   ax=None, alpha=0.75):
        self.true_labels = true_labels
        self.pred_probs = predicted_probs
        self.pred_labels = pred_labels
        self.axes = ax
        df = pd.DataFrame(self.pred_probs)
        df.columns = ['prob_0', 'prob_1']
        df['pred_labels'] = pred_labels
        df['true_labels'] = true_labels
        df.loc[df['true_labels'] == 1, 'user_type'] = "positive_user"
        df.loc[df['true_labels'] == 0, 'user_type'] = "negative_user"

        bins = bins
        if self.axes is None:
            # sns.set_theme("whitegrid")
            self.plot = plt.figure(figsize=[12, 12])
            self.axes = self.plot.add_subplot(1, 1, 1)
            # sns.histplot(x="prob_1", data=df, hue="user_type", bins=bins, ax=self.axes, legend=True)

            self.axes.hist(df[df['true_labels'] == 1].prob_1.values, bins=bins,
                           label="Positive User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)
            self.axes.hist(df[df['true_labels'] == 0].prob_1.values, bins=bins,
                           label="Negative User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)

            self.axes.set_title('Frequency Count Score for ' + self.classifier_name)
            self.axes.set_xlabel('scores')
            self.axes.set_ylabel('Frequency Count')

            return self.plot
        else:
            pass
            # sns.histplot(x="prob_1", data=df, hue="user_type", bins=bins, ax=self.axes, legend=True)

            self.axes.hist(df[df['true_labels'] == 1].prob_1.values, bins=bins, label="Positive User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)
            self.axes.hist(df[df['true_labels'] == 0].prob_1.values, bins=bins, label="Negative User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)

            self.axes.set_title('Frequency Count Score for ' + self.classifier_name)
            self.axes.set_xlabel('scores')
            self.axes.set_ylabel('Frequency Count')

        return




In [ ]:
### save experiment data

def save_experiment_data(player, tml, path):
    if os.path.exists(path) and os.path.isdir(path):
        print("Path exists, will overwrite the path")
    else:
        os.makedirs(path)

    # save timeline data
    tml.save("timeline", path)

    # save decisions
    # player.decision_scores, player.user_profiles
    # put everything into a big dictionary
    json_dict = {
        'user': player.user_profile,
        'decisions': []
    }
    for key, score in player.decision_scores.items():
        json_dict['decisions'].append(
            {
                'strategy': key,
                'result': [
                          {'time': time, 'score': s.tolist()}
                          for time, s in score
                ]
            }
        )

    # print(json_dict)

    with open(os.path.join(path, "result.json"), "w") as outfile:
        json.dump(json_dict, outfile)

    print("file saved.")

In [ ]:
def segment_by_session_end(tml):
    st = 0
    segments = []
    for time, event, content in tml.events:
        if event == EVENT_SESSION_END:
            segments.append((st, time))
            st = time
    return segments

# print(segment_by_session_end(tml))

def gen_decisions_by_segment(tml, player, segments):
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        grt_list = []
        pred_list = []
        time_list = []
        segment_no = 0
        st, ed = segments[segment_no]
        seg_grt = []
        seg_pred = []
        seg_time = []
        for time, result in dec_list:
            user = tml.get_groundtruth_result(time)
            assert time >= st
            if time > ed:
                grt_list.append(seg_grt)
                pred_list.append(seg_pred)
                time_list.append(seg_time)
                seg_grt = []
                seg_pred = []
                seg_time = []
                segment_no += 1
                st, ed = segments[segment_no]
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            seg_grt.append(grt)
            seg_pred.append(result)
            seg_time.append(time - st)
        grt_list.append(seg_grt)
        pred_list.append(seg_pred)
        time_list.append(seg_time)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list,
            'time': time_list
        }
    return result_dict


def get_metrics_for_experiment2(decision_pair, sliding_window=None):
    ## types of errors:
    ## early failures: early rejection and early misclassification
    ## identification failures: 1. false rejection 2. misclassification
    ## there is no negative case

    result_dict = {
        'case': 0,
        'er': 0,
        'em': 0,
        'fr': 0,
        'mi': 0,
        'latency': []
    }

    # owner = decision_pair['true'][0]
    # sharee = decision_pair['true'][-1]

    # moment = 0
    # for i, j in zip(decision_pair['true'], decision_pair['time']):
    #     if i == sharee:
    #         moment = j
    #         break


    for i, j, t in zip(decision_pair['pred'], decision_pair['true'],
                      decision_pair['time']):
        # print(i)
        # print(j)
        score = np.array(i)
        truth = np.array(j)
        # print(score, truth)
        result = np.argmax(score, axis=1)

        # obtain moment:
        owner = truth[0]
        sharee = truth[-1]
        moment = -1
        for i, j in zip(t, truth):
            if j == sharee:
                moment = i
                break
        assert moment != -1

        result_dict['case'] += 1

        if sliding_window is not None:
            m, n = sliding_window
            sw = SlidingWindow(m, n)
            early_rejection = False
            early_misclassification = False
            false_rejection = False
            misclassification = False
            flag = False
            for res, tru, tim in zip(result, truth, t):
                decision = sw.add(res)
                if tru == owner:
                    # pre sharing moment
                    if decision == 0: # false rejection
                        early_rejection = True
                        # print(tim, decision)
                    elif decision != owner and decision > 0:
                        early_misclassification = True
                elif tru == sharee:
                    if decision == 0:
                        false_rejection = True
                    # if decision == owner and :
                    #     misclassification = True
                    if decision != sharee and tim - moment > 60000 and decision > 0:
                        misclassification = True
                    if not flag and decision == sharee:
                        result_dict['latency'].append(tim - moment)
                        flag = True

            if early_rejection:
                result_dict['er'] += 1
            elif early_misclassification:
                result_dict['em'] += 1
            elif false_rejection:
                result_dict['fr'] += 1
            elif misclassification:
                result_dict['mi'] += 1


    return result_dict


def gen_per_segment_decision(tml: Timeline, player: TestPlayer):
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        grt_list = []
        pred_list = []
        time_list = []
        segment_no = 0
        st, ed, user, _ = tml.user_groundtruth[segment_no]
        seg_grt = []
        seg_pred = []
        seg_time = []
        for time, result in dec_list:
            # user = tml.get_groundtruth_result(time)
            assert time >= st
            if time > ed:
                # move to the next segment
                grt_list.append(seg_grt)
                pred_list.append(seg_pred)
                time_list.append(seg_time)
                seg_grt = []
                seg_pred = []
                seg_time = []
                segment_no += 1
                assert segment_no <= len(tml.user_groundtruth)
                st, ed, user, _ = tml.user_groundtruth[segment_no]
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            seg_grt.append(grt)
            seg_pred.append(result)
            seg_time.append(time - st)
        grt_list.append(seg_grt)
        pred_list.append(seg_pred)
        time_list.append(seg_time)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list,
            'time': time_list
        }
    return result_dict



def get_metrics_per_segment(decision_pair, sliding_window=None):
    positives = 0
    negatives = 0
    false_rejections = 0
    true_rejections = 0
    false_identifications = 0
    latency_rejection = []
    result_dict = {
        'pos': 0,
        'neg': 0,
        'fr': 0,
        'tr': 0,
        'fi': 0,
        'latency': []
    }

    for i, j, t in zip(decision_pair['pred'], decision_pair['true'],
                      decision_pair['time']):
        # print(i)
        # print(j)
        score = np.array(i)
        truth = np.array(j)
        # print(score, truth)
        if len(score) == 0:
            continue

        result = np.argmax(score, axis=1)
        if sum(truth) > 0:
            result_dict['pos'] += 1
            # positive case, focus on false rejection and false identification
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res in result:
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif decision != truth[0] and decision > 0:
                        print(decision, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
            else:
                for res in result:
                    if res == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif res != truth[0]:
                        print(res, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
        else:
            result_dict['neg'] += 1
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res, time in zip(result, t):
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
            else:
                for res, time in zip(result, t):
                    if res == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
    return result_dict


In [ ]:
## initialize hmog dataset provider
hmog = HMOGProvider("drive/MyDrive/dataset/HMOG")

# Negative Imposter Set

This function only determines who are the imposter trainers. The amount of the negative data is not determined. It is the developers who determine how much data should be used for training.

In [ ]:
## Negative Trainer (ImposterProvider)
trainer_seed = 100
num_trainers = 10

## Step 1: obtain trainers
trainers = hmog.gen_user_set(num_trainers, random_state=trainer_seed)

## Step 2: load data
hmog.load(trainers)

## Step 3: obtain session data for each authenticator

### Step 3.1: gait
gait_set = []
for user in trainers:
    ## gait sessions
    sel_sessions = hmog.get_sessions_for_unloaded_user(user,
                                [hmog.ACTIVITY_READING_WALKING,
                                 hmog.ACTIVITY_WRITING_WALKING])
    result = None
    for session in sel_sessions:
        tmp_data = simple_merge_acc_gyro(hmog.data[user][session]['acc'],
                                         hmog.data[user][session]['gyro'])
        features = DeepGaitAuthenticator().extract_features(tmp_data)
        if features is None:
            print(user, session, "contains no gait")
            continue
        result = features if result is None else np.concatenate(
            [result, features])
    gait_set.append(result)

### Step 3.2: touch
touch_set = []
for user in trainers:
    ## gait sessions
    sel_sessions = hmog.get_sessions_for_unloaded_user(user,
                                [hmog.ACTIVITY_READING_WALKING,
                                 hmog.ACTIVITY_READING_SITTING])
    result = None
    for session in sel_sessions:
        # print(user, session)
        features = touch_df_to_features(hmog.data[user][session]['touch'])
        # if user == '219303' and int(session) == 17:
        #     print(len(features))
        result = features if result is None else np.concatenate(
            [result, features])
    touch_set.append(result)

## Step4: generate imposter set
gait_imposters = ImposterProvider(gait_set)
gait_train, gait_test = gait_imposters.generate_train_test_sets(2)
touch_imposters = ImposterProvider(touch_set)
touch_train, touch_test = touch_imposters.generate_train_test_sets(2)
# print(gait_train.shape, gait_test.shape, touch_train.shape, touch_test.shape)
imposter_set = {
    'touch': {'train': touch_train, 'test': touch_test},
    'gait': {'train': gait_train, 'test': gait_test}
}

# Load Gait Extraction Model

In [ ]:
dge = tf.keras.models.load_model("drive/MyDrive/deep_gait_extraction_21")

# Experiment 1: Standard Accuracy


In [ ]:
def run_topic_accuracy_experiment_balance_limit(game_seed=0, n_balanced=3,
                                                n_train_session=2,limit=180000):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================

    num_testers = n_balanced
    num_valid_users = n_balanced

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                          str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    for user in valid_users:
        gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

        touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

        enrollment_script_type_1.append((user, 'gait', gait_sessions))
        enrollment_script_type_1.append((user, 'touch', touch_sessions))

    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================


    for user in valid_users:
        sel_session = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[-1]
        events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time,limit)
        tml.add_special_event(EVENT_SESSION_START, [])
        tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
        tml.add_special_event(EVENT_SESSION_END, [])
        # tml.add_special_event(EVENT_TRAIN_MODELS)

    for user in testers:
        sel_session = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[-1]
        events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time,limit)
        tml.add_special_event(EVENT_SESSION_START, [])
        tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
        tml.add_special_event(EVENT_SESSION_END, [])


    # ===============================================
    # 5. One line running script
    # ===============================================

    results = script_runner(tml, player)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    return tml, player

In [ ]:
#@title Experiment 1.0: 1 + 1 training sessions

last_result = None

for seed in range(1000, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 3, 1)
    score_pairs = gen_per_segment_decision(tml, player)
    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/re_accuracy_3_1_limit", str(seed)))

In [ ]:
#@title Experiment 1.1: 2 + 2 training sessions

last_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 3, 2)
    score_pairs = gen_per_segment_decision(tml, player)
    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022/re_accuracy_3_2_limit", str(seed)))

========== SETUP ===========
Game name:100_0_10_3_3
Negative trainer seed:100
Game seed:0
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:554303 (0), 663153 (1), 710707 (2)
testers:171538, 277905, 771782

Load data to memory. It may take up to 5 minutes...
User already registered.
User already registered.
User already registered.
touch
              precision    recall  f1-score   support

           0       0.82      0.75      0.78        24
           1       0.62      0.62      0.62         8
           2       1.00      1.00      1.00         8
           3       0.70      0.88      0.78         8

    accuracy                           0.79        48
   macro avg       0.79      0.81      0.80        48
weighted avg       0.80      0.79      0.79        48

Epoch 1/25
78/78 [==============================] - 5s 19ms/step - loss: 1.2471 - categorical_accuracy: 0.5710
Epoch 2/25
78/78 [==============================] - 2s 19ms/step - los

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Streaming output truncated to the last 5000 lines.
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 29960 ms
simulation result: FAR: 0.0
MIR: 0.0
FNR: 0.03664921465968586
latency: 49517.166666666664 ms
2 3
[touch] pos: 57, neg: 57, fr: 12, tr: 46, fi: 7, 
[gait] pos: 57, neg: 57, fr: 16, tr: 41, fi: 3, 
[simple_average] pos: 57, neg: 57, fr: 20, tr: 48, fi: 6, 
[weighted_average_eer] pos: 57, neg: 57, fr: 18, tr: 46, fi: 6, 
[weighted_average_auc] pos: 57, neg: 57, fr: 20, tr: 48, fi: 6, 
[deer_dempster_shafer] pos: 57, neg: 57, fr: 18, tr: 49, fi: 5, 
[auc_dempster_shafer] pos: 57, neg: 57, fr: 18, tr: 50, fi: 4, 
[kalman_filter] pos: 57, neg: 57, fr: 15, tr: 47, fi: 10, 

file saved.
========== SETUP ===========
Game name:100_1900_10_3_3
Negative trainer seed:100
Game seed:1900
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:796581 (0), 990622 (1), 777078 (2)
testers:986737, 763813, 872895

Load data to memory. It may take up to 5 minutes...
User alre

In [ ]:
#@title Experiment 1.2: 3 + 3 training sessions

last_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 3, 3)
    score_pairs = gen_per_segment_decision(tml, player)
    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022_3/re_accuracy_3_3", str(seed)))

========== SETUP ===========
Game name:100_0_10_3_3
Negative trainer seed:100
Game seed:0
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:554303 (0), 663153 (1), 710707 (2)
testers:171538, 277905, 771782

Load data to memory. It may take up to 5 minutes...
User already registered.
User already registered.
User already registered.
touch
              precision    recall  f1-score   support

           0       0.97      0.85      0.90        39
           1       0.68      1.00      0.81        13
           2       1.00      0.92      0.96        13
           3       0.69      0.69      0.69        13

    accuracy                           0.86        78
   macro avg       0.84      0.87      0.84        78
weighted avg       0.88      0.86      0.86        78

Epoch 1/25
115/115 [==============================] - 6s 19ms/step - loss: 1.0854 - categorical_accuracy: 0.6210
Epoch 2/25
115/115 [==============================] - 2s 18ms/step -

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Streaming output truncated to the last 5000 lines.
==== Session #5 ====
# of detection: 66, acceptance: 0, rejection: 66
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 25687 ms
simulation result: FAR: 0.0
MIR: 0.0
FNR: 0.05190311418685121
latency: 37143.0 ms
2 3
[touch] pos: 57, neg: 57, fr: 12, tr: 46, fi: 8, 
[gait] pos: 57, neg: 57, fr: 18, tr: 46, fi: 1, 
[simple_average] pos: 57, neg: 57, fr: 14, tr: 45, fi: 2, 
[weighted_average_eer] pos: 57, neg: 57, fr: 11, tr: 42, fi: 4, 
[weighted_average_auc] pos: 57, neg: 57, fr: 13, tr: 45, fi: 4, 
[deer_dempster_shafer] pos: 57, neg: 57, fr: 10, tr: 52, fi: 1, 
[auc_dempster_shafer] pos: 57, neg: 57, fr: 12, tr: 51, fi: 1, 
[kalman_filter] pos: 57, neg: 57, fr: 7, tr: 51, fi: 19, 

file saved.
========== SETUP ===========
Game name:100_1900_10_3_3
Negative trainer seed:100
Game seed:1900
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:796581 (0), 990622 (1), 777078 (2)
te

In [ ]:
#@title Experiment 1.3: 5 users 3 + 3 training sessions

last_result = None

for seed in range(1600, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 5, 3)
    score_pairs = gen_per_segment_decision(tml, player)
    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022_3/re_accuracy_5_3_limit", str(seed)))

========== SETUP ===========
Game name:100_1600_10_5_5
Negative trainer seed:100
Game seed:1600
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:785899 (0), 872895 (1), 277905 (2), 472761 (3), 396697 (4)
testers:257279, 171538, 693572, 990622, 980953

Load data to memory. It may take up to 5 minutes...
User already registered.
User already registered.
User already registered.
User already registered.
User already registered.
touch
              precision    recall  f1-score   support

           0       0.96      0.85      0.90        60
           1       1.00      1.00      1.00        12
           2       0.59      0.83      0.69        12
           3       0.60      1.00      0.75        12
           4       0.85      0.92      0.88        12
           5       1.00      0.42      0.59        12

    accuracy                           0.84       120
   macro avg       0.83      0.84      0.80       120
weighted avg       0.88      0.8

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: 

touch
              precision    recall  f1-score   support

           0       0.67      0.78      0.72        60
           1       0.38      0.42      0.40        12
           2       0.00      0.00      0.00        12
           3       0.58      0.58      0.58        12
           4       1.00      0.92      0.96        12
           5       0.64      0.75      0.69        12

    accuracy                           0.66       120
   macro avg       0.55      0.57      0.56       120
weighted avg       0.60      0.66      0.62       120

Epoch 1/25
44/44 [==============================] - 4s 18ms/step - loss: 1.5575 - categorical_accuracy: 0.5163
Epoch 2/25
44/44 [==============================] - 1s 16ms/step - loss: 0.9200 - categorical_accuracy: 0.6647
Epoch 3/25
44/44 [==============================] - 1s 16ms/step - loss: 0.7361 - categorical_accuracy: 0.7409
Epoch 4/25
44/44 [==============================] - 1s 16ms/step - loss: 0.6286 - categorical_accuracy: 0.7627
Epoch 5

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


gait
              precision    recall  f1-score   support

         0.0       0.94      0.77      0.85       305
         1.0       0.60      0.93      0.73        61
         2.0       0.78      0.87      0.82        61
         3.0       0.78      0.87      0.82        61
         4.0       0.97      1.00      0.98        61
         5.0       0.91      0.97      0.94        61

    accuracy                           0.85       610
   macro avg       0.83      0.90      0.86       610
weighted avg       0.87      0.85      0.85       610

training done
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
Not found
==== Session #0 ====
# of detection: 35, acceptance: 4, rejection: 31
# of FA: 0
# of FR: 31
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.8857142857142857
latency: 25765 ms
==== Session #1 ====
# of detection: 50, acceptance: 50, rejection: 0
# of FA: 0
# of F

In [ ]:
#@title Experiment 1.3: 4 users 3 + 3 training sessions

last_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 5, 3)
    score_pairs = gen_per_segment_decision(tml, player)

    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/accuracy_balance_5_3_limit", str(seed)))

In [ ]:
#@title Experiment 1.3: 10 users 3 + 3 training sessions

last_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 10, 3)
    score_pairs = gen_per_segment_decision(tml, player)

    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022_3/re_accuracy_10_3_limit", str(seed)))

Streaming output truncated to the last 5000 lines.
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
Not found
==== Session #0 ====
# of detection: 48, acceptance: 48, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 20399 ms
==== Session #1 ====
# of detection: 46, acceptance: 46, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 35974 ms
==== Session #2 ====
# of detection: 34, acceptance: 34, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 23887 ms
==== Session #3 ====
# of detection: 100, acceptance: 100, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 15541 ms
==== Session #4 ====
# of detection: 9, acceptance: 9, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0

In [ ]:
#@title Experiment 1.3: 7 users 3 + 3 training sessions

last_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_accuracy_experiment_balance_limit(seed, 7, 3)
    score_pairs = gen_per_segment_decision(tml, player)

    result_met = {i: get_metrics_per_segment(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if last_result is None:
        last_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                last_result[i][j] += sc
    to_display = ""
    for key, item in last_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022_3/re_accuracy_7_3_limit", str(seed)))

Streaming output truncated to the last 5000 lines.
==== Session #1 ====
# of detection: 51, acceptance: 50, rejection: 1
# of FA: 0
# of FR: 1
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0196078431372549
latency: 25563 ms
==== Session #2 ====
# of detection: 32, acceptance: 32, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 25962 ms
==== Session #3 ====
# of detection: 35, acceptance: 35, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 25668 ms
==== Session #4 ====
# of detection: 40, acceptance: 40, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 25812 ms
==== Session #5 ====
# of detection: 36, acceptance: 36, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 25667 ms
==== Session #6 ====
# of detection: 38, acceptance: 38, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.0
latency: 31223 ms
==== Session #7 ====
# of detection: 41, accep

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


gait
              precision    recall  f1-score   support

         0.0       0.68      0.37      0.48       427
         1.0       0.91      0.33      0.48        61
         2.0       0.94      1.00      0.97        61
         3.0       0.49      0.82      0.61        61
         4.0       0.60      0.44      0.51        61
         5.0       0.22      0.97      0.36        61
         6.0       0.97      0.46      0.62        61
         7.0       0.67      1.00      0.80        61

    accuracy                           0.54       854
   macro avg       0.68      0.67      0.60       854
weighted avg       0.68      0.54      0.55       854

training done
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: 

gait
              precision    recall  f1-score   support

         0.0       0.68      0.65      0.67       133
         1.0       0.45      0.68      0.54        19
         2.0       0.58      0.37      0.45        19
         3.0       0.95      0.95      0.95        19
         4.0       0.49      0.95      0.64        19
         5.0       0.00      0.00      0.00        19
         6.0       0.26      0.26      0.26        19
         7.0       0.82      0.95      0.88        19

    accuracy                           0.62       266
   macro avg       0.53      0.60      0.55       266
weighted avg       0.59      0.62      0.60       266

training done
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
current session does not belong to a valid user, discarded
Not found
==== Session #0 ====
# of

# Experiment 2: User Switch

## Menu

- 2.1 Basic setting: 2 + 2 for training (```userswitch_3_2```)


In [ ]:
def run_topic_user_switch_experiment_limit(game_seed=0, n_balanced=3,
                                           n_train_session=2, limit=180000):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================

    num_testers = 0
    num_valid_users = n_balanced

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                          str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    for user in valid_users:
        gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

        touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

        enrollment_script_type_1.append((user, 'gait', gait_sessions))
        enrollment_script_type_1.append((user, 'touch', touch_sessions))

    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================

    for m in range(len(valid_users)):
        i = 1
        user_a = valid_users[m]
        for n in range(m + 1, len(valid_users)):
            user_b = valid_users[n]
            if user_a == user_b:
                continue
            sel_session = hmog.get_sessions_for_unloaded_user(
                user_a,[hmog.ACTIVITY_READING_WALKING])[-i]
            events = get_events_for_user_session_with_limit(user_a, sel_session, tml.end_time, limit)
            tml.add_special_event(EVENT_SESSION_START, [])
            tml.add_user_session(user_a, hmog.ACTIVITY_READING_WALKING, events)

            sel_session = hmog.get_sessions_for_unloaded_user(
                user_b,[hmog.ACTIVITY_READING_WALKING])[-i]
            events = get_events_for_user_session_with_limit(user_b, sel_session, tml.end_time, limit)
            tml.add_user_session(user_b, hmog.ACTIVITY_READING_WALKING, events)
            tml.add_special_event(EVENT_SESSION_END, [])
            i = i + 1


    # ===============================================
    # 5. One line running script
    # ===============================================

    results = script_runner(tml, player)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    return tml, player

In [ ]:
#@title Experiment 2.1: 2 + 2 training sessions

experiment_result = None

for seed in range(0, 15000, 100):
    tml, player = run_topic_user_switch_experiment_limit(seed, 3, 2, 180000)
    score_pairs = gen_decisions_by_segment(tml, player, segment_by_session_end(tml))
    result_met = {i: get_metrics_for_experiment2(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if experiment_result is None:
        experiment_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                experiment_result[i][j] += sc
    to_display = ""
    for key, item in experiment_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022_3/user_switch_3_2_limit", str(seed)))

Streaming output truncated to the last 5000 lines.
# of MI: 9
FAR: 0.0
MIR: 0.0891089108910891
FRR: 0.0
latency: 29725 ms
simulation result: FAR: 0.0
MIR: 0.08146067415730338
FNR: 0.0
latency: 22362.666666666668 ms
[touch] case: 6, er: 2, em: 0, fr: 1, mi: 0, 
[gait] case: 6, er: 0, em: 0, fr: 2, mi: 1, 
[simple_average] case: 6, er: 0, em: 0, fr: 2, mi: 3, 
[weighted_average_eer] case: 6, er: 0, em: 0, fr: 1, mi: 4, 
[weighted_average_auc] case: 6, er: 0, em: 0, fr: 1, mi: 4, 
[deer_dempster_shafer] case: 6, er: 0, em: 0, fr: 0, mi: 5, 
[auc_dempster_shafer] case: 6, er: 1, em: 0, fr: 0, mi: 5, 
[kalman_filter] case: 6, er: 0, em: 2, fr: 0, mi: 3, 

file saved.
========== SETUP ===========
Game name:100_200_10_0_3
Negative trainer seed:100
Game seed:200
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:923862 (0), 538363 (1), 808022 (2)
testers:

Load data to memory. It may take up to 5 minutes...
User already registered.
User already regist

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


gait
              precision    recall  f1-score   support

         0.0       0.93      0.64      0.76        42
         1.0       0.78      1.00      0.88        14
         2.0       0.93      0.93      0.93        14
         3.0       0.52      0.86      0.65        14

    accuracy                           0.79        84
   macro avg       0.79      0.86      0.80        84
weighted avg       0.84      0.79      0.79        84

training done
Not found
==== Session #0 ====
# of detection: 67, acceptance: 67, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 4
FAR: 0.0
MIR: 0.05970149253731343
FRR: 0.0
latency: 62345 ms
==== Session #1 ====
# of detection: 72, acceptance: 45, rejection: 27
# of FA: 0
# of FR: 27
# of MI: 7
FAR: 0.0
MIR: 0.09722222222222222
FRR: 0.375
latency: 52101 ms
==== Session #2 ====
# of detection: 81, acceptance: 59, rejection: 22
# of FA: 0
# of FR: 22
# of MI: 2
FAR: 0.0
MIR: 0.024691358024691357
FRR: 0.2716049382716049
latency: 25695 ms
simulation result: FAR

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


gait
              precision    recall  f1-score   support

         0.0       0.81      0.62      0.70        42
         1.0       0.62      0.57      0.59        14
         2.0       0.54      0.93      0.68        14
         3.0       0.93      1.00      0.97        14

    accuracy                           0.73        84
   macro avg       0.73      0.78      0.74        84
weighted avg       0.75      0.73      0.73        84

training done
Not found
==== Session #0 ====
# of detection: 115, acceptance: 61, rejection: 54
# of FA: 0
# of FR: 54
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.46956521739130436
latency: 82270 ms
==== Session #1 ====
# of detection: 70, acceptance: 70, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 7
FAR: 0.0
MIR: 0.1
FRR: 0.0
latency: 25691 ms
==== Session #2 ====
# of detection: 99, acceptance: 55, rejection: 44
# of FA: 0
# of FR: 44
# of MI: 0
FAR: 0.0
MIR: 0.0
FRR: 0.4444444444444444
latency: 42461 ms
simulation result: FAR: 0.0
MIR: 0.02464788732394366
FNR

...

gait
              precision    recall  f1-score   support

         0.0       1.00      0.71      0.83       417
         1.0       0.63      0.99      0.77       139
         2.0       0.74      0.99      0.85       139
         3.0       1.00      0.99      1.00       139

    accuracy                           0.85       834
   macro avg       0.84      0.92      0.86       834
weighted avg       0.89      0.85      0.85       834

training done
Not found
==== Session #0 ====
# of detection: 77, acceptance: 77, rejection: 0
# of FA: 0
# of FR: 0
# of MI: 8
FAR: 0.0
MIR: 0.1038961038961039
FRR: 0.0
latency: 25962 ms
==== Session #1 ====
# of detection: 79, acceptance: 50, rejection: 29
# of FA: 0
# of FR: 29
# of MI: 8
FAR: 0.0
MIR: 0.10126582278481013
FRR: 0.3670886075949367
latency: 31100 ms
==== Session #2 ====
# of detection: 76, acceptance: 71, rejection: 5
# of FA: 0
# of FR: 5
# of MI: 9
FAR: 0.0
MIR: 0.11842105263157894
FRR: 0.06578947368421052
latency: 25734 ms
simulation r

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Streaming output truncated to the last 5000 lines.
# of FA: 0
# of FR: 0
# of MI: 13
FAR: 0.0
MIR: 0.10569105691056911
FRR: 0.0
latency: 20624 ms
simulation result: FAR: 0.0
MIR: 0.26571428571428574
FNR: 0.10857142857142857
latency: 87437.66666666667 ms
[touch] case: 309, er: 56, em: 17, fr: 62, mi: 24, 
[gait] case: 309, er: 91, em: 19, fr: 63, mi: 11, 
[simple_average] case: 309, er: 76, em: 23, fr: 42, mi: 97, 
[weighted_average_eer] case: 309, er: 61, em: 29, fr: 38, mi: 104, 
[weighted_average_auc] case: 309, er: 69, em: 26, fr: 40, mi: 102, 
[deer_dempster_shafer] case: 309, er: 64, em: 19, fr: 32, mi: 110, 
[auc_dempster_shafer] case: 309, er: 68, em: 18, fr: 38, mi: 104, 
[kalman_filter] case: 309, er: 67, em: 56, fr: 31, mi: 96, 

file saved.
========== SETUP ===========
Game name:100_10300_10_0_3
Negative trainer seed:100
Game seed:10300
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:366286 (0), 657486 (1), 151985 (2)
testers:

L

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


gait
              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90        42
         1.0       0.86      0.86      0.86        14
         2.0       0.88      1.00      0.93        14
         3.0       1.00      0.86      0.92        14

    accuracy                           0.90        84
   macro avg       0.91      0.90      0.90        84
weighted avg       0.91      0.90      0.90        84

training done
Not found
==== Session #0 ====
# of detection: 65, acceptance: 53, rejection: 12
# of FA: 0
# of FR: 12
# of MI: 8
FAR: 0.0
MIR: 0.12307692307692308
FRR: 0.18461538461538463
latency: 53407 ms
==== Session #1 ====
# of detection: 89, acceptance: 67, rejection: 22
# of FA: 0
# of FR: 22
# of MI: 5
FAR: 0.0
MIR: 0.056179775280898875
FRR: 0.24719101123595505
latency: 52916 ms
==== Session #2 ====
# of detection: 72, acceptance: 22, rejection: 50
# of FA: 0
# of FR: 50
# of MI: 3
FAR: 0.0
MIR: 0.041666666666666664
FRR: 0.6944444444444444
latency

In [ ]:
#@title Experiment 2.1: 2 + 2 + 30 training sessions

experiment_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_user_switch_experiment_limit(seed, 3, 2, 180000)
    score_pairs = gen_decisions_by_segment(tml, player, segment_by_session_end(tml))
    result_met = {i: get_metrics_for_experiment2(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if experiment_result is None:
        experiment_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                experiment_result[i][j] += sc
    to_display = ""
    for key, item in experiment_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/user_switch_3_2_30_limit", str(seed)))

In [ ]:
#@title Experiment 2.1: 3 + 3 training sessions

experiment_result = None

for seed in range(9200, 15000, 100):
    tml, player = run_topic_user_switch_experiment_limit(seed, 3, 3, 180000)
    score_pairs = gen_decisions_by_segment(tml, player, segment_by_session_end(tml))
    result_met = {i: get_metrics_for_experiment2(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if experiment_result is None:
        experiment_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                experiment_result[i][j] += sc
    to_display = ""
    for key, item in experiment_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022/user_switch_3_3_limit", str(seed)))

========== SETUP ===========
Game name:100_9200_10_0_3
Negative trainer seed:100
Game seed:9200
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:827212 (0), 472761 (1), 777078 (2)
testers:

Load data to memory. It may take up to 5 minutes...
User already registered.
User already registered.
User already registered.
touch
              precision    recall  f1-score   support

           0       0.96      0.89      0.92        27
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         9
           3       0.73      0.89      0.80         9

    accuracy                           0.93        54
   macro avg       0.92      0.94      0.93        54
weighted avg       0.93      0.93      0.93        54

Epoch 1/25
26/26 [==============================] - 3s 18ms/step - loss: 1.8739 - categorical_accuracy: 0.4306
Epoch 2/25
26/26 [==============================] - 0s 17ms/step - loss: 1.0222 - cate

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Streaming output truncated to the last 5000 lines.
Epoch 7/25
103/103 [==============================] - 2s 17ms/step - loss: 0.2743 - categorical_accuracy: 0.8899
Epoch 8/25
103/103 [==============================] - 2s 16ms/step - loss: 0.2508 - categorical_accuracy: 0.9014
Epoch 9/25
103/103 [==============================] - 2s 17ms/step - loss: 0.2300 - categorical_accuracy: 0.9085
Epoch 10/25
103/103 [==============================] - 2s 17ms/step - loss: 0.1947 - categorical_accuracy: 0.9249
Epoch 11/25
103/103 [==============================] - 2s 17ms/step - loss: 0.1847 - categorical_accuracy: 0.9265
Epoch 12/25
103/103 [==============================] - 2s 17ms/step - loss: 0.1578 - categorical_accuracy: 0.9372
Epoch 13/25
103/103 [==============================] - 2s 17ms/step - loss: 0.1275 - categorical_accuracy: 0.9524
Epoch 14/25
103/103 [==============================] - 2s 17ms/step - loss: 0.1100 - categorical_accuracy: 0.9589
Epoch 15/25
103/103 [===================

In [ ]:
#@title Experiment 2.1: 10 users 2 + 2 training sessions

experiment_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_user_switch_experiment_limit(seed, 10, 2, 180000)
    score_pairs = gen_decisions_by_segment(tml, player, segment_by_session_end(tml))
    result_met = {i: get_metrics_for_experiment2(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if experiment_result is None:
        experiment_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                experiment_result[i][j] += sc
    to_display = ""
    for key, item in experiment_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/user_switch_10_2_limit", str(seed)))

In [ ]:
#@title Experiment 2.1: 5 users 2 + 2 training sessions

experiment_result = None

for seed in range(0, 5000, 100):
    tml, player = run_topic_user_switch_experiment_limit(seed, 5, 2, 180000)
    score_pairs = gen_decisions_by_segment(tml, player, segment_by_session_end(tml))
    result_met = {i: get_metrics_for_experiment2(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if experiment_result is None:
        experiment_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                experiment_result[i][j] += sc
    to_display = ""
    for key, item in experiment_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/user_switch_5_2_limit", str(seed)))

In [ ]:
def run_topic_switch_attack_experiment_limit(game_seed=0, n_balanced=3,
                                           n_train_session=2, limit=180000):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================

    num_testers = n_balanced
    num_valid_users = n_balanced

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                          str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    for user in valid_users:
        gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

        touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

        enrollment_script_type_1.append((user, 'gait', gait_sessions))
        enrollment_script_type_1.append((user, 'touch', touch_sessions))

    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================

    for m in range(len(valid_users)):
        user_a = valid_users[m]
        user_b = testers[m]
        # for n in range(m + 1, len(valid_users)):
        #     user_b = valid_users[n]
        #     if user_a == user_b:
        #         continue
        #     sel_session = hmog.get_sessions_for_unloaded_user(
        #         user_a,[hmog.ACTIVITY_READING_WALKING])[-i]
        #     events = get_events_for_user_session_with_limit(user_a, sel_session, tml.end_time, limit)
        #     tml.add_special_event(EVENT_SESSION_START, [])
        #     tml.add_user_session(user_a, hmog.ACTIVITY_READING_WALKING, events)

        #     sel_session = hmog.get_sessions_for_unloaded_user(
        #         user_b,[hmog.ACTIVITY_READING_WALKING])[-i]
        #     events = get_events_for_user_session_with_limit(user_b, sel_session, tml.end_time, limit)
        #     tml.add_user_session(user_b, hmog.ACTIVITY_READING_WALKING, events)
        #     tml.add_special_event(EVENT_SESSION_END, [])
        #     i = i + 1
        sel_session = hmog.get_sessions_for_unloaded_user(
            user_a,[hmog.ACTIVITY_READING_WALKING])[-1]
        events = get_events_for_user_session_with_limit(user_a, sel_session, tml.end_time, limit)
        tml.add_special_event(EVENT_SESSION_START, [])
        tml.add_user_session(user_a, hmog.ACTIVITY_READING_WALKING, events)

        sel_session = hmog.get_sessions_for_unloaded_user(
            user_b,[hmog.ACTIVITY_READING_WALKING])[-1]
        events = get_events_for_user_session_with_limit(user_b, sel_session, tml.end_time, limit)
        tml.add_user_session(user_b, hmog.ACTIVITY_READING_WALKING, events)
        tml.add_special_event(EVENT_SESSION_END, [])


    # ===============================================
    # 5. One line running script
    # ===============================================

    results = script_runner(tml, player)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    return tml, player

In [ ]:
#@title Experiment 2.2: 2 + 2 training sessions

experiment_result = None

for seed in range(0, 15000, 100):
    tml, player = run_topic_switch_attack_experiment_limit(seed, 3, 3, 180000)
    score_pairs = gen_decisions_by_segment(tml, player, segment_by_session_end(tml))
    result_met = {i: get_metrics_for_experiment2(score_pairs[i], (7, 4))
                  for i in score_pairs.keys()}
    if experiment_result is None:
        experiment_result = result_met
    else:
        for i, score in result_met.items():
            for j, sc in score.items():
                experiment_result[i][j] += sc
    to_display = ""
    for key, item in experiment_result.items():
        to_display += "[" + key + "] "
        for metr, value in item.items():
            if metr == 'latency':
                continue
            to_display += metr + ": " + str(value) + ", "
        to_display += "\r\n"
    print(to_display)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results2022/attack_switch_3_3_limit", str(seed)))

Streaming output truncated to the last 5000 lines.
file saved.
========== SETUP ===========
Game name:100_1600_10_3_3
Negative trainer seed:100
Game seed:1600
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:913228 (0), 986737 (1), 277905 (2)
testers:257279, 171538, 693572

Load data to memory. It may take up to 5 minutes...
User already registered.
User already registered.
User already registered.
touch
              precision    recall  f1-score   support

           0       0.97      0.84      0.90        45
           1       0.82      0.93      0.87        15
           2       0.79      1.00      0.88        15
           3       1.00      1.00      1.00        15

    accuracy                           0.91        90
   macro avg       0.90      0.94      0.92        90
weighted avg       0.92      0.91      0.91        90

Epoch 1/25
9/9 [==============================] - 4s 20ms/step - loss: 2.9455 - categorical_accuracy: 0.3722
Epo

# Experiment 3: Robustness (Data and class incremental)

In [ ]:
def run_topic_robustness_type_cmp(game_seed=0):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================


    num_testers = 0
    num_valid_users = 3

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "robust" + "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                                     str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)


    player2 = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)
    player2.set_flag_store_history(False) # player2 never learn

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    # for user in valid_users:
    #     gait_sessions = hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

    #     touch_sessions = hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

    #     enrollment_script_type_1.append((user, 'gait', gait_sessions))
    #     enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR OWNER
    user = valid_users[0]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:2]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:2] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR VALID USER 1
    user = valid_users[1]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))

    tf.random.set_seed(game_seed)
    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")

    tf.random.set_seed(game_seed)
    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player2.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player2.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player2.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================


    np.random.seed(0)
    lengths = np.random.randint(120000,300000, 20)


    # Part 1: owner - user 1

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[0])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[1])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    # Part 2: new user
    user = valid_users[2]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    user_info = [user]
    for session in gait_sessions:
        user_info += ['gait', session]
    for session in touch_sessions :
        user_info += ['touch', session]
    tml.add_special_event(EVENT_ADD_USER, user_info)

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[2])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[3])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[4])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    ## Part 3 New data
    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[5])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[6])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[7])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])


    # for user_a in valid_users:
    #     i = 1
    #     for user_b in valid_users:
    #         if user_a == user_b:
    #             continue
    #         sel_session = hmog.get_sessions_for_unloaded_user(
    #             user_a,[hmog.ACTIVITY_READING_WALKING])[-i]
    #         events = get_events_for_user_session(user_a, sel_session, tml.end_time)
    #         tml.add_special_event(EVENT_SESSION_START, [])
    #         tml.add_user_session(user_a, hmog.ACTIVITY_READING_WALKING, events)

    #         sel_session = hmog.get_sessions_for_unloaded_user(
    #             user_b,[hmog.ACTIVITY_READING_WALKING])[-i]
    #         events = get_events_for_user_session(user_b, sel_session, tml.end_time)
    #         tml.add_user_session(user_b, hmog.ACTIVITY_READING_WALKING, events)
    #         # tml.add_special_event(EVENT_SESSION_END)
    #         # tml.add_special_event(EVENT_TRAIN_MODELS)
    #         tml.add_special_event(EVENT_SESSION_END, [])
    #         i = i + 1


    # ===============================================
    # 5. One line running script
    # ===============================================
    np.random.seed(game_seed)
    tf.random.set_seed(game_seed)
    results = script_runner(tml, player)
    np.random.seed(game_seed)
    tf.random.set_seed(game_seed)
    results2 = script_runner(tml, player2)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    print("===> simulator 1 <===")
    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    print("===> simulator 2 <===")

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    for i, result in enumerate(results2):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)


    return tml, player, player2

In [ ]:
def run_topic_robustness_type_1(game_seed=0):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================

    num_testers = 0
    num_valid_users = 3

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "robust" + "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                                     str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    # for user in valid_users:
    #     gait_sessions = hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

    #     touch_sessions = hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

    #     enrollment_script_type_1.append((user, 'gait', gait_sessions))
    #     enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR OWNER
    user = valid_users[0]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:2]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:2] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR VALID USER 1
    user = valid_users[1]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))



    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================


    # Part 1: owner - user 1

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    # Part 2: new user
    user = valid_users[2]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    user_info = [user]
    for session in gait_sessions:
        user_info += ['gait', session]
    for session in touch_sessions :
        user_info += ['touch', session]
    tml.add_special_event(EVENT_ADD_USER, user_info)

    np.random.seed(seed)
    lengths = np.random.randint(120000,300000, 10) # 2 to 5 minutes

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    ## Part 3 New data
    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session(user, sel_session, tml.end_time)
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])


    # for user_a in valid_users:
    #     i = 1
    #     for user_b in valid_users:
    #         if user_a == user_b:
    #             continue
    #         sel_session = hmog.get_sessions_for_unloaded_user(
    #             user_a,[hmog.ACTIVITY_READING_WALKING])[-i]
    #         events = get_events_for_user_session(user_a, sel_session, tml.end_time)
    #         tml.add_special_event(EVENT_SESSION_START, [])
    #         tml.add_user_session(user_a, hmog.ACTIVITY_READING_WALKING, events)

    #         sel_session = hmog.get_sessions_for_unloaded_user(
    #             user_b,[hmog.ACTIVITY_READING_WALKING])[-i]
    #         events = get_events_for_user_session(user_b, sel_session, tml.end_time)
    #         tml.add_user_session(user_b, hmog.ACTIVITY_READING_WALKING, events)
    #         # tml.add_special_event(EVENT_SESSION_END)
    #         # tml.add_special_event(EVENT_TRAIN_MODELS)
    #         tml.add_special_event(EVENT_SESSION_END, [])
    #         i = i + 1


    # ===============================================
    # 5. One line running script
    # ===============================================

    results = script_runner(tml, player)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    return tml, player

In [ ]:
for i in range(0, 5000, 100):
    tml, player = run_topic_robustness_type_1(i)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_new_1", str(i)))

In [ ]:
for i in range(2000, 5000, 100):
    tml, player = run_topic_robustness_type_1(i)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_1", str(i)))

In [ ]:
def run_topic_robustness_type_2(game_seed=0):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================

    num_testers = 0
    num_valid_users = 3

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "robust" + "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                                     str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    }, imposter_set=imposter_set)

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    # for user in valid_users:
    #     gait_sessions = hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

    #     touch_sessions = hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

    #     enrollment_script_type_1.append((user, 'gait', gait_sessions))
    #     enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR OWNER
    user = valid_users[0]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:2]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:2] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR VALID USER 1
    user = valid_users[1]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))



    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_session_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================

    np.random.seed(seed)
    lengths = np.random.randint(120000,300000, 20)


    # Part 1: owner - user 1

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[0])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[1])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    # Part 2: new user
    user = valid_users[2]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    user_info = [user]
    for session in gait_sessions:
        user_info += ['gait', session]
    for session in touch_sessions :
        user_info += ['touch', session]
    tml.add_special_event(EVENT_ADD_USER, user_info)

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[2])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[3])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[4])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    ## Part 3 New data
    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[5])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[6])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[7])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])


    # for user_a in valid_users:
    #     i = 1
    #     for user_b in valid_users:
    #         if user_a == user_b:
    #             continue
    #         sel_session = hmog.get_sessions_for_unloaded_user(
    #             user_a,[hmog.ACTIVITY_READING_WALKING])[-i]
    #         events = get_events_for_user_session(user_a, sel_session, tml.end_time)
    #         tml.add_special_event(EVENT_SESSION_START, [])
    #         tml.add_user_session(user_a, hmog.ACTIVITY_READING_WALKING, events)

    #         sel_session = hmog.get_sessions_for_unloaded_user(
    #             user_b,[hmog.ACTIVITY_READING_WALKING])[-i]
    #         events = get_events_for_user_session(user_b, sel_session, tml.end_time)
    #         tml.add_user_session(user_b, hmog.ACTIVITY_READING_WALKING, events)
    #         # tml.add_special_event(EVENT_SESSION_END)
    #         # tml.add_special_event(EVENT_TRAIN_MODELS)
    #         tml.add_special_event(EVENT_SESSION_END, [])
    #         i = i + 1


    # ===============================================
    # 5. One line running script
    # ===============================================

    results = script_runner(tml, player)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    return tml, player

In [ ]:
for i in range(10000, 15000, 100):
    tml, player = run_topic_robustness_type_2(i)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_new_2", str(i)))

In [ ]:
for i in range(0, 10000, 100):
    tml, player = run_topic_robustness_type_2(i)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_new_2", str(i)))

In [ ]:
for i in range(9400, 10000, 100):
    tml, player, player2 = run_topic_robustness_type_cmp(i)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_new_cmp_1", str(i)))
    save_experiment_data(player2, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_new_cmp_2", str(i)))

In [ ]:
for i in range(8000, 10000, 100):
    tml, player, player2 = run_topic_robustness_type_cmp(i)
    save_experiment_data(player, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_cmp_2_1", str(i)))
    save_experiment_data(player2, tml, os.path.join("/content/drive/MyDrive/muia_experiment_results/robustness_cmp_2_2", str(i)))